## Download HF dataset

In [ ]:
import pandas as pd

n_per_dataset = 200
df = pd.DataFrame(columns=['prompt_en', 'prompt_ar', 'response_en', 'response_ar', 'source'])

In [ ]:
from datasets import load_dataset

dataset_repo = 'akoksal/LongForm'
dataset = load_dataset(dataset_repo, split='test')

df_longForm = pd.DataFrame(dataset)
df_longForm.shape

In [ ]:
df_longForm.head()

In [ ]:
subsampled_df = df_longForm[['input', 'output']].sample(n=n_per_dataset).reset_index(drop=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df['input'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df['output'].tolist(),
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'mlabonne/alpagasus'
dataset = load_dataset(dataset_repo, split='train')

df_alpagasus = pd.DataFrame(dataset)
df_alpagasus.shape

In [ ]:
df_alpagasus = df_alpagasus[df_alpagasus['input'] == '']
df_alpagasus.shape

In [ ]:
df_alpagasus.head()

In [ ]:
subsampled_df = df_alpagasus[['instruction', 'output']].sample(n=n_per_dataset).reset_index(drop=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df['instruction'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df['output'].tolist(),
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'HuggingFaceH4/Koala-test-set'
dataset = load_dataset(dataset_repo, split='test')

df_koala = pd.DataFrame(dataset)
df_koala.shape

In [ ]:
df_koala.head()

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': df_koala['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': '',
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'HuggingFaceH4/ultrachat_200k'
dataset = load_dataset(dataset_repo, split='test_sft')

df_ultra = pd.DataFrame(dataset)
df_ultra.shape

In [ ]:
df_ultra.head()

In [ ]:
df_ultra.messages.iloc[0][1]

In [ ]:
subsampled_df = df_ultra[['prompt', 'messages']].sample(n=n_per_dataset).reset_index(drop=True)
collect_prompt = []
collect_response = []

for i in range(subsampled_df.shape[0]):
    messages = subsampled_df.messages.iloc[i]
    current_role = 'user'

    collect_prompt_sub = []
    collect_response_sub = []
    
    for message in messages:
        role = message['role']
        content = message['content']
        if role != current_role:
            break
        
        if current_role == 'user':
            collect_prompt_sub.append(content)
            current_role = 'assistant'
        else:
            collect_response_sub.append(content)
            current_role = 'user'

    if len(collect_prompt_sub) != 0:
        collect_prompt.append(collect_prompt_sub)
        collect_response.append(collect_response_sub)

len(collect_prompt), len(collect_response)

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': collect_prompt,
     'prompt_ar': '',
     'response_en': collect_response,
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'HuggingFaceH4/mt_bench_prompts'
dataset = load_dataset(dataset_repo, split='train')

df_mt = pd.DataFrame(dataset)
df_mt.shape

In [ ]:
df_mt.head()

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': df_mt['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': '',
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'reciprocate/alpaca-eval'
dataset = load_dataset(dataset_repo, split='test')

df_alpaca = pd.DataFrame(dataset)
df_alpaca.shape

In [ ]:
df_alpaca.head()

In [ ]:
subsampled_df = df_alpaca[['prompt', 'selected']].sample(n=n_per_dataset).reset_index(drop=True)

subsampled_df['prompt'] = subsampled_df['prompt'].str.lstrip('USER: ')
subsampled_df['prompt'] = subsampled_df['prompt'].str.rstrip(' ASSISTANT: ')

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': [i for i in subsampled_df['prompt'].tolist()],
     'prompt_ar': '',
     'response_en': subsampled_df['selected'].tolist(),
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'natolambert/xstest-v2-copy'
dataset = load_dataset(dataset_repo, split='gpt4')

df_xstest = pd.DataFrame(dataset)
df_xstest.shape

In [ ]:
df_xstest.head()

In [ ]:
df_xstest.type.value_counts()

In [ ]:
df_xstest['focus_type'] = df_xstest.type.apply(lambda x: 'contrast' if 'contrast' in x else 'others')

In [ ]:
df_xstest.focus_type.value_counts()

In [ ]:
subsampled_df_contrast = df_xstest[df_xstest.focus_type == 'contrast']
subsampled_df_others = df_xstest[df_xstest.focus_type != 'contrast']

subsampled_df_contrast = subsampled_df_contrast[['prompt', 'completion']].sample(n=n_per_dataset//2).reset_index(drop=True)
subsampled_df_others = subsampled_df_others[['prompt', 'completion']].sample(n=n_per_dataset//2).reset_index(drop=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df_contrast['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df_contrast['completion'].tolist(),
     'response_ar': '',
     'source': f'{dataset_repo}_contrast',
    })], ignore_index=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df_others['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df_others['completion'].tolist(),
     'response_ar': '',
     'source': f'{dataset_repo}_others',
    })], ignore_index=True)

df.shape

In [ ]:
df_personal = pd.read_csv('red-team-personal.csv')
df_personal.shape

In [ ]:
df_personal.head()

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': df_personal['PROMPT'].tolist(),
     'prompt_ar': '',
     'response_en': '',
     'response_ar': '',
     'source': f'fanar_redteam',
    })], ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.source.value_counts()

In [ ]:
df.to_csv('assets/analyzeDataset.csv', index=False)

### Translate content to arabic

In [ ]:
import pandas as pd
df = pd.read_csv('assets/analyzeDataset.csv')
df.shape

In [ ]:
df.source.value_counts()

In [ ]:
df.head()

In [ ]:
from utils import get_correct_format

df.prompt_en = df.prompt_en.apply(get_correct_format)
df.response_en = df.response_en.apply(get_correct_format)

In [ ]:
from utils import google_translator

google_translator(df.prompt_en.iloc[0])

In [ ]:
from tqdm import tqdm

cnt_exception = 0

for i in tqdm(range(df.shape[0])):
    prompt = df.prompt_en.iloc[i]
    try:
        if isinstance(prompt, list):
            prompt_tmp = []
            for p in prompt:
                prompt_tmp.append(google_translator(p))
            df.at[i, 'prompt_ar'] = prompt_tmp
        else:
            df.at[i, 'prompt_ar'] = google_translator(prompt)
    
        response = df.response_en.iloc[i]
        if isinstance(response, list):
            response_tmp = []
            for r in response:
                response_tmp.append(google_translator(r))
            df.at[i, 'response_ar'] = response_tmp
        else:
            df.at[i, 'response_ar'] = google_translator(response)
    except:
        cnt_exception += 1

    if not i%10:
        df.to_csv('assets/analyzeDataset.csv', index=False)

df.head()

In [ ]:
df.to_csv('assets/analyzeDataset.csv', index=False)

In [ ]:
df.head()

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'lmsys/vicuna-7b-v1.5'
quant_path = 'vicuna-7b-v1.5-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage": True})
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

/home/local/QCRI/ejeong/anaconda2/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: libcudart.so.11.0: cannot open shared object file: No such file or directory

In [5]:
from vllm import LLM, SamplingParams

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Create an LLM.
llm = LLM(model="lmsys/vicuna-7b-v1.5")
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

INFO 04-23 15:23:31 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='lmsys/vicuna-7b-v1.5', tokenizer='lmsys/vicuna-7b-v1.5', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 04-23 15:23:36 weight_utils.py:177] Using model weights format ['*.bin']
INFO 04-23 15:40:42 model_runner.py:104] Loading model weights took 12.5523 GB


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
